# Purpose
- This file is a scratch-pad for sql queries

In [1]:
%load_ext sql
%sql duckdb:///../data/referral.db --alias referralsDB

In [2]:
%sql referralsDB

In [3]:
%%sql 
CREATE OR REPLACE TABLE team as 

SELECT DISTINCT * 

FROM read_csv(
    '../data/reference/referral_team.csv'
    , delim=',' 
    , header=True
    , columns={'Names': 'VARCHAR', 'Team': 'VARCHAR'}
)

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
16


In [ ]:
%sql CREATE OR REPLACE TABLE hcpcs as SELECT DISTINCT * FROM read_csv_auto('../data/reference/cpt_hcpcs.csv', header=True)

In [ ]:
%sql CREATE OR REPLACE TABLE icd10cm as SELECT DISTINCT * FROM read_csv('../data/reference/icd10cm.csv', header=True, columns={'code_value': 'VARCHAR', 'code_long_description': 'VARCHAR'})

In [ ]:
%%sql 
CREATE OR REPLACE TABLE cln_referrals as 

    SELECT DISTINCT * 
    FROM read_parquet('../data/clean_referrals/*.parquet')

In [ ]:
%sql DESCRIBE cln_referrals

- No Auth Required ->	Appointment Scheduled or Member to Schedule or Unable to Contact
- Approved	       ->   Appointment Scheduled or Member to Schedule or Unable to Contact
- Denied	       ->   Open
- Withdrawn	       ->   Open

In [4]:
%%sql 
CREATE OR REPLACE TABLE bi_referrals as

    SELECT DISTINCT *
    , COALESCE(tm.Team, 'N/A') as User_Team

    , UPPER("Last Name") || UPPER("First Name") as patient_name

    , UPPER("Last Name") || UPPER("First Name") || "DOB" as patient_id

    , patient_id || coalesce("Plan ID", 'NA') || coalesce("Referring Provider NPI", 'NA') || "Referral Date" || coalesce("Procedure", 'NA') || coalesce(Specialty, 'NA') as Referral_keyid

    , CASE
        WHEN "Approval Status" IN ('No HP Auth Required', 'HP Approved', 'Approved (comments required)', 'Complete/no Auth# needed')  
            AND "Visit Status" IN ('Appointment Scheduled', 'Member to Schedule', 'Unable to Contact', 'Denied', 'Withdrawn')
            THEN 'Completed'
        WHEN "Approval Status" IN ('Denied- Per Insurance Plan', 'Denied- Per Medical Director Review')
            AND "Visit Status" = 'Open'
            THEN 'Completed'
        ELSE "Referral Status" END as "Updated Status"

    , RIGHT("Procedure", 5) as Proc_code
    , pc.code_short_description as Proc_name 

    , FORMAT(
        '({}) {}-{}'
        , substring("Mobile Phone", 1, 3), substring("Mobile Phone", 3, 3), substring("Mobile Phone", 6, 4)
    ) as "Fmt Mobile Phone"

    , FORMAT(
        '({}) {}-{}'
        , substring("Home Phone", 1, 3), substring("Home Phone", 3, 3), substring("Home Phone", 6, 4)
    ) as "Fmt Home Phone"

    , icd.code_long_description AS "Diagnosis Description"

    FROM cln_referrals AS rf

    LEFT JOIN icd10cm AS icd
        ON trim(rf."Diagnosis") = trim(icd.code_value)

    LEFT JOIN hcpcs AS pc
        on RIGHT("Procedure", 5) = pc.code_value

    LEFT JOIN team as tm
        on "Last Update" = tm.Names

    WHERE "Visit Status" IS NOT NULL

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
58663


In [5]:
%sql SELECT DISTINCT * FROM bi_referrals WHERE User_Team != '' LIMIT 25

*  (referralsDB) duckdb:///../data/referral.db
Done.


C0,Center,Clinic,Referring Provider,Referring Provider NPI,Referral Date,Last Name,First Name,Sex,DOB,pat Status,Language,Address,Mobile Phone,Home Phone,PCP,Referred to Specialist,Specialist NPI,Referral Status,Network,Specialty,Diagnosis,Procedure,Approval Status,Visit Status,Referral Type,Patient Type,Health Plan,Plan ID,LOB,Auth No,Last Update,Update_DT,file_source,code_value,code_long_description,code_short_description,code_value:1,code_category,Names,Team,User_Team,patient_name,patient_id,Referral_keyid,Updated Status,Proc_code,Proc_name,Fmt Mobile Phone,Fmt Home Phone,Diagnosis Description
1513,"Centrum Medical Holdings, LLC",CH - Coral Springs,"Katherine Perez, MD",1164487427,04-05-23,HENAO,ANA,F,02-07-61,Active,Spanish,"4113 Riverside DrCoral Springs, FL, 33065",7542147104,7542147104,"Perez, Katherine",None,None,Open,Out of network,Emergency Medicine,R42,EMERGENCY DEPT VISIT HIGH SEVERITY&THREAT FUNCJ - 99285,HP Approved,Pending Appt,External,Adult,Ambetter (Centrum Contract),U9694113901,Exchange,N/A,"Soria, Jaqueline",Apr 5 2023 9:38AM,../data/raw_referrals/excels/FLORIDA REFERRALS 3_31-4_6.xlsx,R42,DIZZINESS AND GIDDINESS,EMERGENCY DEPT VISIT,99285,CPT,None,None,N/A,HENAOANA,HENAOANA02-07-61,HENAOANA02-07-61U9694113901116448742704-05-23EMERGENCY DEPT VISIT HIGH SEVERITY&THREAT FUNCJ - 99285Emergency Medicine,Open,99285,EMERGENCY DEPT VISIT,(754) 421-4710,(754) 421-4710,DIZZINESS AND GIDDINESS
13116,"Centrum Medical Holdings, LLC",Home Visit,"Yuralis Santamaria, APRN",1982256889,03-15-23,FERIA,IRMA,F,09-16-69,Active,Spanish,"9000 ROYAL PALM BLVD APT E303CORAL SPRINGS, FL, 33065",9546955593,9546955593,"Perez, Katherine",Coral Spring Medical Center. Michael J Boyle MD,None,Open,Out of network,General Surgery,N60.02,BX BREAST NEEDLE CORE W/O IMAGING GUIDANCE SPX - 19100,No HP Auth Required,Appointment Scheduled,External,Adult,Simply (Medicaid) Med-Care,729204304,Medicaid,N/A,"Gutierrez, Jenny",Mar 28 2023 2:33PM,../data/raw_referrals/excels/FL REFERRALS 3_1-3_31.xlsx,N60.02,SOLITARY CYST OF LEFT BREAST,BX BREAST PERCUT W/O IMAGE,19100,CPT,None,None,N/A,FERIAIRMA,FERIAIRMA09-16-69,FERIAIRMA09-16-69729204304198225688903-15-23BX BREAST NEEDLE CORE W/O IMAGING GUIDANCE SPX - 19100General Surgery,Completed,19100,BX BREAST PERCUT W/O IMAGE,(954) 469-5559,(954) 469-5559,SOLITARY CYST OF LEFT BREAST
12131,"Centrum Medical Holdings, LLC",Home Visit,"Yuralis Santamaria, APRN",1982256889,3/15/2023,FERIA,IRMA,F,9/16/1969,Active,Spanish,"9000 ROYAL PALM BLVD APT E303CORAL SPRINGS, FL, 33065",9546955593,9546955593,"Perez, Katherine",Coral Spring Medical Center. Michael J Boyle MD,None,Open,Out of network,General Surgery,N60.02,BX BREAST NEEDLE CORE W/O IMAGING GUIDANCE SPX - 19100,No HP Auth Required,Appointment Scheduled,External,Adult,Simply (Medicaid) Med-Care,729204304,Medicaid,N/A,"Gutierrez, Jenny",Mar 28 2023 2:33PM,data/raw_referrals/FL REFERRALS 3.1-3.29.CSV,N60.02,SOLITARY CYST OF LEFT BREAST,BX BREAST PERCUT W/O IMAGE,19100,CPT,None,None,N/A,FERIAIRMA,FERIAIRMA9/16/1969,FERIAIRMA9/16/196972920430419822568893/15/2023BX BREAST NEEDLE CORE W/O IMAGING GUIDANCE SPX - 19100General Surgery,Completed,19100,BX BREAST PERCUT W/O IMAGE,(954) 469-5559,(954) 469-5559,SOLITARY CYST OF LEFT BREAST
1384,Centrum Health - Texas,(H) EASTEX FREEWAY,"Emilia Okere, APRN",1760983886,3/2/2023,salinas,edward,M,6/19/1962,Active,English,"8906 lockwood drhouston, TX, 77016",7133068312,7133068312,"Inyang, Magdalene",SIMONMED IMAGING,None,Open,Out of network,Diagnostic Testing,M25.562,RADIOLOGIC EXAMINATION KNEE 3 VIEWS - 73562,Complete/no Auth# needed,Pending,External,Adult,Ambetter Superior Health- Exchange,U9697889201,Exchange,N/A,"Miller, Kalie",Mar 2 2023 5:29PM,data/raw_referrals/TX REFERRALS 3.1-3.29.CSV,M25.562,PAIN IN LEFT KNEE,X-RAY EXAM OF KNEE 3,73562,CPT,None,None,N/A,SALINASEDWARD,SALINASEDWARD6/19/1962,SALINASEDWARD6/19/1962U969788920117609838863/2/2023RADIOLOGIC EXAMINATION KNEE 3 VIEWS - 73562Diagnostic Testing,Open,73562,X-RAY EXAM O

In [ ]:
%%sql 
COPY (SELECT * FROM bi_referrals) TO '../data/report_data/2023-04-06.parquet' (FORMAT 'parquet'); 